In [1]:
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
import pandas as pd
import re
import nltk


df = pd.read_csv("../data/recipes.csv")


lemmatizer = WordNetLemmatizer()
smart_stopwords = set(stopwords.words('english')).union({
    "cup", "cups", "teaspoon", "tablespoon", "ounce", "pound", "gram", "slice", "can", "jar", "package", "pinch", "dash", "clove", "large", "medium", "small", "fresh", "dry", "chopped", "ground", "patted", "quality", "kosher", "virgin", "sun", "taste", "room", "temperature", "part", "accompaniment", "thermometer",
})

def preprocess_ingredients_smart(ingredients_list_str):

    try:
        if isinstance(ingredients_list_str, str):
            ingredients = ingredients_list_str.strip("[]").replace("'", "").split(', ')
        elif isinstance(ingredients_list_str, list):
            ingredients = ingredients_list_str
        else:
            return []
    except:
        return []

    cleaned_tokens = []
    for item in ingredients:
        text = item.lower()
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub(r'[\d½¾¼⅓⅔⅛⅜⅝⅞]+', '', text)
        text = re.sub(r'[^a-z\s]', ' ', text)

        tokens = nltk.word_tokenize(text)
        tagged_tokens = nltk.pos_tag(tokens)

        for word, tag in tagged_tokens:
            if tag.startswith('NN'):
                lemma = lemmatizer.lemmatize(word)
                if lemma not in smart_stopwords and len(lemma) > 2:
                    cleaned_tokens.append(lemma)

    return cleaned_tokens

In [2]:
try:
    df = pd.read_csv("../data/recipes.csv")
    print("✅ Daten erfolgreich geladen!")
    print(f"Datensatz hat {df.shape[0]} Zeilen und {df.shape[1]} Spalten.")
except FileNotFoundError:
    print("⚠️ FEHLER: Die CSV-Datei wurde nicht gefunden. Bitte Pfad überprüfen.")

✅ Daten erfolgreich geladen!
Datensatz hat 13501 Zeilen und 6 Spalten.


In [3]:
#nltk.download('punkt_tab')
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('averaged_perceptron_tagger_eng')



print("Starte smartes Preprocessing mit POS-Tagging...")

df['ingredients_smart'] = df['Ingredients'].apply(preprocess_ingredients_smart)

print("✅ Smartes Preprocessing abgeschlossen!")

# --- Zeige ein Beispiel, um das Ergebnis zu prüfen ---
print("\n--- Beispiel-Ergebnis des smarten Preprocessing ---")
for i in range(3):
    print(f"RAW: {df['Ingredients'].iloc[i][:100]}...") # Nur die ersten 100 Zeichen
    print(f"SMART: {df['ingredients_smart'].iloc[i]}")
    print("-" * 20)

Starte smartes Preprocessing mit POS-Tagging...
✅ Smartes Preprocessing abgeschlossen!

--- Beispiel-Ergebnis des smarten Preprocessing ---
RAW: ['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more', '2 small acorn squash (abo...
SMART: ['chicken', 'tsp', 'salt', 'acorn', 'squash', 'tbsp', 'sage', 'tbsp', 'tbsp', 'butter', 'tsp', 'allspice', 'pepper', 'flake', 'pepper', 'bread', 'piece', 'apple', 'cut', 'piece', 'tbsp', 'oil', 'onion', 'tbsp', 'apple', 'cider', 'vinegar', 'miso', 'flour', 'tbsp', 'butter', 'wine', 'broth', 'miso', 'salt', 'pepper']
--------------------
RAW: ['2 large egg whites', '1 pound new potatoes (about 1 inch in diameter)', '2 teaspoons kosher salt',...
SMART: ['egg', 'white', 'potato', 'salt', 'pepper', 'thyme', 'parsley']
--------------------
RAW: ['1 cup evaporated milk', '1 cup whole milk', '1 tsp. garlic powder', '1 tsp. onion powder', '1 tsp....
SMART: ['milk', 'milk', 'tsp', 'powder', 'onion', 'powder', 'tsp', 'paprika', 'tsp', 'pepper',

In [4]:
from gensim.models.phrases import Phraser
from gensim.models import Phrases

print("Starte Training des Bigramm-Modells...")

# Trainiere das Phrases-Modell auf den neuen, smarten Daten
phrases_model_smart = Phrases(df['ingredients_smart'], min_count=5, threshold=0.1, scoring="npmi")
bigram_model_smart = Phraser(phrases_model_smart)

# Wende das Bigramm-Modell an
df['ingredients_bigrams_smart'] = df['ingredients_smart'].apply(lambda tokens: bigram_model_smart[tokens])

print("✅ Bigramm-Modell trainiert und angewendet!")

# --- Zeige ein Beispiel für die Bigramm-Erkennung ---
print("\n--- Beispiel für erkannte Bigramme ---")
# Finde ein paar Beispiele, wo sich etwas geändert hat
for i in range(len(df)):
    if df['ingredients_smart'].iloc[i] != df['ingredients_bigrams_smart'].iloc[i]:
        print(f"Vorher: {df['ingredients_smart'].iloc[i]}")
        print(f"Nachher: {df['ingredients_bigrams_smart'].iloc[i]}")
        print("-" * 30)
        break # Zeige nur das erste Beispiel

Starte Training des Bigramm-Modells...
✅ Bigramm-Modell trainiert und angewendet!

--- Beispiel für erkannte Bigramme ---
Vorher: ['chicken', 'tsp', 'salt', 'acorn', 'squash', 'tbsp', 'sage', 'tbsp', 'tbsp', 'butter', 'tsp', 'allspice', 'pepper', 'flake', 'pepper', 'bread', 'piece', 'apple', 'cut', 'piece', 'tbsp', 'oil', 'onion', 'tbsp', 'apple', 'cider', 'vinegar', 'miso', 'flour', 'tbsp', 'butter', 'wine', 'broth', 'miso', 'salt', 'pepper']
Nachher: ['chicken', 'tsp_salt', 'acorn', 'squash', 'tbsp', 'sage', 'tbsp', 'tbsp_butter', 'tsp_allspice', 'pepper_flake', 'pepper', 'bread', 'piece', 'apple_cut', 'piece', 'tbsp_oil', 'onion', 'tbsp', 'apple_cider', 'vinegar_miso', 'flour', 'tbsp_butter', 'wine', 'broth', 'miso', 'salt_pepper']
------------------------------


In [5]:
import os
import multiprocessing
from gensim.models import Word2Vec

print("Starte finales Word2Vec-Training...")

cores = multiprocessing.cpu_count()

# Trainiere das Word2Vec-Modell auf den smarten Bigrammen
model_smart = Word2Vec(df['ingredients_bigrams_smart'],
                       workers=cores-1,
                       vector_size=150, # Erhöhen wir mal die Vektorgröße für potenziell bessere Ergebnisse
                       window=10,       # und das Fenster
                       min_count=5,
                       sg=1)           # Skip-Gram

# Erstelle den Ordner, falls er nicht existiert
os.makedirs("../data/models", exist_ok=True)

# Speichere das Modell unter einem neuen, klaren Namen
model_path_smart = "../data/models/recipe_word2vec_smart.model"
model_smart.save(model_path_smart)

print(f"✅ Smartes Modell trainiert und gespeichert unter: {model_path_smart}")

Starte finales Word2Vec-Training...


Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


✅ Smartes Modell trainiert und gespeichert unter: ../data/models/recipe_word2vec_smart.model


In [6]:
# Lade das gerade gespeicherte Modell, um sicherzugehen, dass alles geklappt hat
loaded_model_smart = Word2Vec.load(model_path_smart)

def check_smart(term, model):
    """Eine Funktion, um die Ähnlichkeit von Wörtern zu prüfen."""
    try:
        similar = model.wv.most_similar(term, topn=5)
        print(f"\nAlternativen zu '{term}' (SMART-MODELL):")
        for item, score in similar:
            print(f"  -> {item} ({score:.2f})")
    except KeyError:
        print(f"\n❌ '{term}' kenne ich im smarten Modell nicht.")

# Teste das Modell mit relevanten Begriffen
check_smart("chicken", loaded_model_smart)
check_smart("beef", loaded_model_smart)
check_smart("chocolate", loaded_model_smart)
check_smart("spaghetti", loaded_model_smart)
check_smart("tomato", loaded_model_smart) # Teste ein Bigramm!
check_smart("oil", loaded_model_smart)     # Und noch eins!


Alternativen zu 'chicken' (SMART-MODELL):
  -> breast (0.86)
  -> leg_thigh (0.86)
  -> wing (0.85)
  -> bone (0.84)
  -> neck (0.84)

Alternativen zu 'beef' (SMART-MODELL):
  -> tomato_sauce (0.93)
  -> turkey (0.91)
  -> beef_chuck (0.90)
  -> beef_pork (0.89)
  -> onion_stalk (0.89)

Alternativen zu 'chocolate' (SMART-MODELL):
  -> bittersweet_chocolate (0.96)
  -> milk_chocolate (0.95)
  -> bittersweet_semisweet (0.95)
  -> semisweet_bittersweet (0.95)
  -> bar (0.95)

Alternativen zu 'spaghetti' (SMART-MODELL):
  -> pasta_salt (0.96)
  -> bucatini (0.95)
  -> penne (0.95)
  -> soppressata (0.94)
  -> rigatoni (0.94)

Alternativen zu 'tomato' (SMART-MODELL):
  -> oregano (0.91)
  -> eggplant (0.91)
  -> onion_tomato (0.87)
  -> oregano_pepper (0.87)
  -> cheese_mozzarella (0.87)

Alternativen zu 'oil' (SMART-MODELL):
  -> pepper_oil (0.77)
  -> oil_salt (0.77)
  -> togarashi (0.76)
  -> parsley_mint (0.76)
  -> paprika_powder (0.75)
